In [ ]:
import pandas as pd

%run '../data-cleaning/data_cleaning_trees.ipynb'
from data_cleaning_trees import df

In [ ]:
#Round up latitude and longtitude to define approximate fields
df['lat'] = df['lat'].round(1)
df['lon'] = df['lon'].round(1)

#In every location, group by lon, lat, and name to see how many of each tree type present
groupedDf = df.groupby(['lon','lat','scientificName'])['scientificName'].count()
groupedDf = groupedDf.rename("count").reset_index()

#Group counts by tree types to make smoothing separately for each type
multiple_lists = [x for _, x in groupedDf.groupby('scientificName')]
rolling_window = []

final_df = pd.DataFrame()

for i in range(0, 11):
  #Sort values according to lon and lat to get closer points in an order
  multiple_lists[i] = multiple_lists[i].sort_values(['lon', 'lat'])

  #Remove other columns so rolling window work
  rolling_window.append(multiple_lists[i][{'count'}])

  #Rolling window with size 5 for each type
  rolling_window[i] = rolling_window[i].rolling(window=5, closed='both').mean()

  #Adding lon, lat, and name back
  rolling_window[i]['lon'] = multiple_lists[i]['lon']
  rolling_window[i]['lat'] = multiple_lists[i]['lat']
  rolling_window[i]['scientificName'] = multiple_lists[i]['scientificName']

  #Round zero because there can't be decimal observations
  rolling_window[i]['count'] = rolling_window[i]['count'].round(0)

  #Create the final data frame
  final_df = final_df.append(rolling_window[i])

final_df

In [ ]:
#NOT FINISHED

#Calculating biodiversity indexes

#Simpson's Index

#Group by lon and lat, and sum count
total_tree_df = final_df[{'lon', 'lat', 'count'}].groupby(['lon','lat'])['count'].sum().reset_index()

#Loop over all elements and calculate ratio
ratio_df = pd.DataFrame()
for i in final_df:
  line = total_tree_df.loc[(total_tree_df['lon'] == i[0]) & (total_tree_df['lat'] == i[1])]
  #ratio = i[0] / line['count']
  print((line))
  ratio_df = ratio_df.append(pd.DataFrame([i[0], i[1], ratio]))

total_tree_df
